In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import random
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import os
from PIL import Image 
import tensorflow as tf
import keras.backend as K
from keras.applications import Xception
from keras.layers import UpSampling2D, Conv2D, Activation, LeakyReLU, BatchNormalization
from keras import Model
from keras.losses import binary_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from tqdm import tqdm_notebook

Using TensorFlow backend.


### The Data

In [2]:
df_train = pd.read_csv('./input/severstal-steel-defect-detection/train.csv')

In [3]:
df_train['Defect'] = df_train['EncodedPixels'].notnull()
df_train['ClassId'] = df_train['ImageId_ClassId'].str[-1:]
df_train['ImageId'] = df_train['ImageId_ClassId'].str[:-2]

df_train = df_train[['ImageId','ClassId','Defect','EncodedPixels']]

In [4]:
df_train = df_train[df_train.Defect]
df_train = df_train.drop(['Defect'], axis=1)

In [5]:
df_train.shape

(7095, 3)

In [6]:
df_train.head()

,ImageId,ClassId,EncodedPixels
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
18,0007a71bf.jpg,3,18661 28 18863 82 19091 110 19347 110 19603 11...
20,000a4bcdd.jpg,1,37607 3 37858 8 38108 14 38359 20 38610 25 388...
27,000f6bf48.jpg,4,131973 1 132228 4 132483 6 132738 8 132993 11 ...
30,0014fce06.jpg,3,229501 11 229741 33 229981 55 230221 77 230468...


### Utility functions

In [7]:
# This function uses a 0 based start index (arrays, matrixes and imgs start a index 0)
def mask2rle(img):
    tmp = np.rot90( np.flipud( img ), k=3 )
    rle = []
    lastColor = 0;
    startpos = 0
    endpos = 0

    tmp = tmp.reshape(-1,1)   
    for i in range( len(tmp) ):
        if (lastColor==0) and tmp[i]>0:
            startpos = i
            lastColor = 1
        elif (lastColor==1)and(tmp[i]==0):
            endpos = i-1
            lastColor = 0
            rle.append( str(startpos)+' '+str(endpos-startpos+1) )
    return " ".join(rle)

In [8]:
img = [
    [0, 1, 0, 0, 0],
    [0, 1, 1, 1, 0], 
    [0, 1, 0, 1, 0],
    [0, 1, 1, 1, 0], 
    [0, 0, 0, 1, 0],
]

res = mask2rle(img)
print(res)

5 4 11 1 13 1 16 4


In [9]:
# This function uses a 0 based start index (arrays, matrixes and imgs start a index 0)
def rle2mask(rle, width, height):
    width = width
    height= height
    
    mask= np.zeros( width*height ).astype(np.uint8)
    
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]

    current_position = 0
    for index, start in enumerate(starts):
        mask[int(start):int(start+lengths[index])] = 1
        current_position += lengths[index]
        
    return np.flipud( np.rot90( mask.reshape(height, width), k=1 ) )

In [10]:
res = rle2mask("5 4 11 1 13 1 16 4", 5, 5)
print(res)

[[0 1 0 0 0]
 [0 1 1 1 0]
 [0 1 0 1 0]
 [0 1 1 1 0]
 [0 0 0 1 0]]


### Xception Model in Keras

In [11]:
ORIG_IMG_SIZE_HEIGHT = 256
ORIG_IMG_SIZE_WIDTH = 1600
TRAIN_IMG_SIZE_HEIGHT = 256
TRAIN_IMG_SIZE_WIDTH = 256

NO_OF_CLASSES = 4

BATCH_SIZE = 4


In [12]:
import keras

class DataGenerator(keras.utils.Sequence):
    def __init__(self, df, img_path, orig_img_shape, new_img_shape=None, batch_size = BATCH_SIZE, n_channels=1,
                 n_classes=4, shuffle=False):
        super().__init__()
        self.df = df
        self.img_path = img_path
        self.orig_img_shape = orig_img_shape
        if new_img_shape:
            self.new_img_shape = new_img_shape
        else:
            self.new_img_shape = orig_img_shape
        self.batch_size = batch_size
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(len(self.df) / self.batch_size))
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    
    def __getitem__(self, index): 
        X = np.empty((self.batch_size, *self.new_img_shape, self.n_channels), dtype=np.uint8)
        y = np.empty((self.batch_size, *self.new_img_shape, 1), dtype=np.uint8)
        
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        for idx, filename in enumerate(self.df['ImageId'].iloc[indexes]):
            img = cv2.imread(self.img_path + filename, cv2.IMREAD_UNCHANGED)
            if self.orig_img_shape != self.new_img_shape:
                img = cv2.resize(img, tuple(reversed(self.new_img_shape)), interpolation=cv2.INTER_AREA)
            X[idx,] = img
            
            mask_i = rle2mask(self.df['EncodedPixels'].iloc[indexes[idx]], *self.orig_img_shape)            
            img_mask_i = Image.fromarray(mask_i, 'L')
            img_mask_np = np.array(img_mask_i) 
            
            if self.orig_img_shape != self.new_img_shape:
                img_mask_np = cv2.resize(img_mask_np, tuple(reversed(self.new_img_shape)), interpolation=cv2.INTER_AREA)
            y[idx,:,:,0] = img_mask_np
            
        return X, y

In [13]:
def dice_coeff(y_true, y_pred):
    smooth = 1
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def bce_dice_loss(y_true, y_pred):
    loss = 0.5 * binary_crossentropy(y_true, y_pred) - 2*dice_coeff(y_true, y_pred)
    return loss


In [14]:
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Activation, UpSampling2D, BatchNormalization

def basic_unet_256_segmentation_model(input_shape=(256, 256, 3), num_classes=1):
    inputs = Input(shape=input_shape) # 256

    down0 = Conv2D(32, (3, 3), padding='same')(inputs)
    down0 = BatchNormalization()(down0)
    down0 = Activation('relu')(down0)
    down0_pool = MaxPooling2D((2, 2), strides=(2, 2))(down0) #128

    down1 = Conv2D(128, (3, 3), padding='same')(down0_pool)
    down1 = BatchNormalization()(down1)
    down1 = Activation('relu')(down1)
    down1_pool = MaxPooling2D((2, 2), strides=(2, 2))(down1) # 64

    down2 = Conv2D(512, (3, 3), padding='same')(down1_pool)
    down2 = BatchNormalization()(down2)
    down2 = Activation('relu')(down2)
    down2_pool = MaxPooling2D((2, 2), strides=(2, 2))(down2) # 32

    down3 = Conv2D(1048, (3, 3), padding='same')(down2_pool)
    down3 = BatchNormalization()(down3)
    down3 = Activation('relu')(down3)
    down3_pool = MaxPooling2D((2, 2), strides=(2, 2))(down3) # 16

    center = Conv2D(2048, (3, 3), padding='same')(down3_pool)
    center = BatchNormalization()(center)
    center = Activation('relu')(center) # center

    up3 = UpSampling2D((2, 2))(center)
    up3 = concatenate([down3, up3], axis=3)
    up3 = Conv2D(1048, (3, 3), padding='same')(up3)
    up3 = BatchNormalization()(up3)
    up3 = Activation('relu')(up3) # 32

    up2 = UpSampling2D((2, 2))(up3)
    up2 = concatenate([down2, up2], axis=3)
    up2 = Conv2D(512, (3, 3), padding='same')(up2)
    up2 = BatchNormalization()(up2)
    up2 = Activation('relu')(up2) # 64

    up1 = UpSampling2D((2, 2))(up2)
    up1 = concatenate([down1, up1], axis=3)
    up1 = Conv2D(128, (3, 3), padding='same')(up1)
    up1 = BatchNormalization()(up1)
    up1 = Activation('relu')(up1) # 128

    up0 = UpSampling2D((2, 2))(up1)
    up0 = concatenate([down0, up0], axis=3)
    up0 = Conv2D(32, (3, 3), padding='same')(up0)
    up0 = BatchNormalization()(up0)
    up0 = Activation('relu')(up0) # 256

    classify = Conv2D(num_classes, (1, 1), activation='sigmoid')(up0)

    model = Model(inputs=inputs, outputs=classify)

    return model

In [15]:
basic_model = basic_unet_256_segmentation_model()
basic_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 256, 32) 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 256, 256, 32) 0           batch_normalization_1[0][0]      
_____________________________________

                                                                 up_sampling2d_4[0][0]            
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 256, 256, 32) 46112       concatenate_4[0][0]              
__________________________________________________________________________________________________
batch_normalization_9 (BatchNor (None, 256, 256, 32) 128         conv2d_9[0][0]                   
__________________________________________________________________________________________________
activation_9 (Activation)       (None, 256, 256, 32) 0           batch_normalization_9[0][0]      
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 256, 256, 1)  33          activation_9[0][0]               
Total params: 61,974,257
Trainable params: 61,963,281
Non-trainable params: 10,976
__________________________

In [16]:
from keras.optimizers import RMSprop

basic_model.compile(optimizer=RMSprop(lr=0.0003), loss=bce_dice_loss, metrics=[dice_coeff, 'accuracy'])

In [17]:
# A decreasing learning rate schedule (continuous decrease)
def continuous_decay(epoch):
    initial_lrate = 0.01
    lr = initial_lrate * (0.1 ** int(epoch / 10))
    return lr

# Another decreasing learning rate schedule (stepped decrease)
def step_decay(epoch):
    initial_lrate = 0.01
    drop = 0.5
    epochs_drop = 5.0
    lr = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lr

def get_callback(patient):
    ES = EarlyStopping(
        monitor='loss', 
        patience=patient, 
        mode='max', 
        verbose=1)
    RR = ReduceLROnPlateau(
        monitor = 'loss', 
        factor = 0.5, 
        patience = patient / 2, 
        min_lr=0.000001, 
        verbose=1, 
        mode='max')
    LR = LearningRateScheduler(
        continuous_decay,
        verbose=1)
    MC = ModelCheckpoint(
        filepath='./Xcep_model_CD_LR0_01.h5', 
        save_best_only=True,
        verbose=1)
#    return [ES, RR, LR, MC]
#    return [ES, MC, RR]
    return [ES, MC, LR]


In [18]:
idx = int(0.8*len(df_train))
train_batches = DataGenerator(df_train.iloc[:idx], './input/severstal-steel-defect-detection/train_images/', 
                                    orig_img_shape=(256, 1600), new_img_shape=(TRAIN_IMG_SIZE_HEIGHT, TRAIN_IMG_SIZE_WIDTH), batch_size = BATCH_SIZE, 
                                    n_channels=3, n_classes=4, shuffle=False)
valid_batches = DataGenerator(df_train.iloc[idx:], './input/severstal-steel-defect-detection/train_images/', 
                                    orig_img_shape=(256, 1600), new_img_shape=(TRAIN_IMG_SIZE_HEIGHT, TRAIN_IMG_SIZE_WIDTH), batch_size = BATCH_SIZE, 
                                    n_channels=3, n_classes=4, shuffle=False)

history = basic_model.fit_generator(train_batches,
                              epochs=3,    
                              validation_data=valid_batches,
                              verbose=1,
                              shuffle=True,
                              callbacks = get_callback(2)
                             )

Instructions for updating:
Use tf.cast instead.
Epoch 1/3

Epoch 00001: LearningRateScheduler setting learning rate to 0.01.


 132/1419 [=>............................] - ETA: 3:36:24 - loss: 0.3377 - dice_coeff: 0.0561 - acc: 0.221 - ETA: 1:53:06 - loss: 0.0931 - dice_coeff: 0.1451 - acc: 0.549 - ETA: 1:18:47 - loss: 0.1304 - dice_coeff: 0.1293 - acc: 0.625 - ETA: 1:01:33 - loss: 0.1337 - dice_coeff: 0.1150 - acc: 0.677 - ETA: 51:09 - loss: 0.1137 - dice_coeff: 0.1119 - acc: 0.6874  - ETA: 44:18 - loss: 0.1084 - dice_coeff: 0.0995 - acc: 0.736 - ETA: 39:23 - loss: 0.0925 - dice_coeff: 0.0960 - acc: 0.769 - ETA: 35:40 - loss: 0.0880 - dice_coeff: 0.0916 - acc: 0.773 - ETA: 32:50 - loss: 0.0747 - dice_coeff: 0.0914 - acc: 0.792 - ETA: 30:31 - loss: 0.0706 - dice_coeff: 0.0857 - acc: 0.811 - ETA: 28:39 - loss: 0.0583 - dice_coeff: 0.0859 - acc: 0.824 - ETA: 27:02 - loss: 0.0566 - dice_coeff: 0.0825 - acc: 0.832 - ETA: 25:45 - loss: 0.0389 - dice_coeff: 0.0883 - acc: 0.840 - ETA: 24:35 - loss: 0.0164 - dice_coeff: 0.0956 - acc: 0.847 - ETA: 23:36 - loss: 0.0134 - dice_coeff: 0.0951 - acc: 0.849 - ETA: 22:45 - lo

 264/1419 [====>.........................] - ETA: 10:34 - loss: -0.2914 - dice_coeff: 0.2201 - acc: 0.90 - ETA: 10:33 - loss: -0.2947 - dice_coeff: 0.2221 - acc: 0.90 - ETA: 10:32 - loss: -0.2973 - dice_coeff: 0.2231 - acc: 0.90 - ETA: 10:31 - loss: -0.3004 - dice_coeff: 0.2244 - acc: 0.90 - ETA: 10:29 - loss: -0.3010 - dice_coeff: 0.2245 - acc: 0.90 - ETA: 10:28 - loss: -0.3017 - dice_coeff: 0.2247 - acc: 0.90 - ETA: 10:27 - loss: -0.3015 - dice_coeff: 0.2243 - acc: 0.90 - ETA: 10:26 - loss: -0.3017 - dice_coeff: 0.2243 - acc: 0.90 - ETA: 10:25 - loss: -0.3036 - dice_coeff: 0.2251 - acc: 0.90 - ETA: 10:24 - loss: -0.3024 - dice_coeff: 0.2244 - acc: 0.90 - ETA: 10:23 - loss: -0.2998 - dice_coeff: 0.2237 - acc: 0.90 - ETA: 10:22 - loss: -0.2995 - dice_coeff: 0.2237 - acc: 0.90 - ETA: 10:21 - loss: -0.2995 - dice_coeff: 0.2239 - acc: 0.90 - ETA: 10:20 - loss: -0.3018 - dice_coeff: 0.2248 - acc: 0.90 - ETA: 10:19 - loss: -0.3035 - dice_coeff: 0.2258 - acc: 0.90 - ETA: 10:18 - loss: -0.304

 396/1419 [=======>......................] - ETA: 8:51 - loss: -0.4322 - dice_coeff: 0.2890 - acc: 0.913 - ETA: 8:51 - loss: -0.4326 - dice_coeff: 0.2891 - acc: 0.913 - ETA: 8:50 - loss: -0.4320 - dice_coeff: 0.2888 - acc: 0.913 - ETA: 8:49 - loss: -0.4325 - dice_coeff: 0.2888 - acc: 0.913 - ETA: 8:49 - loss: -0.4316 - dice_coeff: 0.2883 - acc: 0.913 - ETA: 8:48 - loss: -0.4325 - dice_coeff: 0.2887 - acc: 0.913 - ETA: 8:48 - loss: -0.4340 - dice_coeff: 0.2895 - acc: 0.913 - ETA: 8:47 - loss: -0.4364 - dice_coeff: 0.2905 - acc: 0.914 - ETA: 8:46 - loss: -0.4369 - dice_coeff: 0.2907 - acc: 0.914 - ETA: 8:46 - loss: -0.4391 - dice_coeff: 0.2917 - acc: 0.914 - ETA: 8:45 - loss: -0.4412 - dice_coeff: 0.2926 - acc: 0.914 - ETA: 8:45 - loss: -0.4419 - dice_coeff: 0.2928 - acc: 0.914 - ETA: 8:44 - loss: -0.4407 - dice_coeff: 0.2921 - acc: 0.914 - ETA: 8:43 - loss: -0.4402 - dice_coeff: 0.2918 - acc: 0.915 - ETA: 8:43 - loss: -0.4416 - dice_coeff: 0.2924 - acc: 0.915 - ETA: 8:42 - loss: -0.4429

 528/1419 [==========>...................] - ETA: 7:39 - loss: -0.5181 - dice_coeff: 0.3274 - acc: 0.922 - ETA: 7:39 - loss: -0.5191 - dice_coeff: 0.3279 - acc: 0.922 - ETA: 7:38 - loss: -0.5199 - dice_coeff: 0.3282 - acc: 0.922 - ETA: 7:38 - loss: -0.5199 - dice_coeff: 0.3282 - acc: 0.922 - ETA: 7:37 - loss: -0.5196 - dice_coeff: 0.3283 - acc: 0.922 - ETA: 7:37 - loss: -0.5216 - dice_coeff: 0.3292 - acc: 0.922 - ETA: 7:36 - loss: -0.5226 - dice_coeff: 0.3298 - acc: 0.922 - ETA: 7:36 - loss: -0.5249 - dice_coeff: 0.3308 - acc: 0.923 - ETA: 7:35 - loss: -0.5252 - dice_coeff: 0.3309 - acc: 0.923 - ETA: 7:35 - loss: -0.5250 - dice_coeff: 0.3312 - acc: 0.922 - ETA: 7:34 - loss: -0.5250 - dice_coeff: 0.3311 - acc: 0.923 - ETA: 7:34 - loss: -0.5242 - dice_coeff: 0.3308 - acc: 0.923 - ETA: 7:33 - loss: -0.5239 - dice_coeff: 0.3305 - acc: 0.923 - ETA: 7:33 - loss: -0.5245 - dice_coeff: 0.3308 - acc: 0.923 - ETA: 7:32 - loss: -0.5245 - dice_coeff: 0.3309 - acc: 0.923 - ETA: 7:32 - loss: -0.5243

...

 924/1419 [==================>...........] - ETA: 4:27 - loss: -0.9233 - dice_coeff: 0.5150 - acc: 0.951 - ETA: 4:27 - loss: -0.9233 - dice_coeff: 0.5150 - acc: 0.951 - ETA: 4:26 - loss: -0.9229 - dice_coeff: 0.5149 - acc: 0.951 - ETA: 4:26 - loss: -0.9235 - dice_coeff: 0.5151 - acc: 0.951 - ETA: 4:26 - loss: -0.9238 - dice_coeff: 0.5152 - acc: 0.951 - ETA: 4:25 - loss: -0.9235 - dice_coeff: 0.5151 - acc: 0.951 - ETA: 4:25 - loss: -0.9232 - dice_coeff: 0.5150 - acc: 0.951 - ETA: 4:24 - loss: -0.9238 - dice_coeff: 0.5152 - acc: 0.951 - ETA: 4:24 - loss: -0.9238 - dice_coeff: 0.5152 - acc: 0.951 - ETA: 4:23 - loss: -0.9235 - dice_coeff: 0.5150 - acc: 0.951 - ETA: 4:23 - loss: -0.9234 - dice_coeff: 0.5150 - acc: 0.951 - ETA: 4:23 - loss: -0.9233 - dice_coeff: 0.5150 - acc: 0.951 - ETA: 4:22 - loss: -0.9234 - dice_coeff: 0.5150 - acc: 0.951 - ETA: 4:22 - loss: -0.9239 - dice_coeff: 0.5153 - acc: 0.951 - ETA: 4:21 - loss: -0.9239 - dice_coeff: 0.5153 - acc: 0.951 - ETA: 4:21 - loss: -0.9242

1063/1419 [=====================>........] - ETA: 3:31 - loss: -0.9265 - dice_coeff: 0.5161 - acc: 0.951 - ETA: 3:30 - loss: -0.9259 - dice_coeff: 0.5158 - acc: 0.951 - ETA: 3:30 - loss: -0.9261 - dice_coeff: 0.5158 - acc: 0.951 - ETA: 3:30 - loss: -0.9269 - dice_coeff: 0.5162 - acc: 0.951 - ETA: 3:29 - loss: -0.9275 - dice_coeff: 0.5165 - acc: 0.951 - ETA: 3:29 - loss: -0.9277 - dice_coeff: 0.5166 - acc: 0.951 - ETA: 3:28 - loss: -0.9274 - dice_coeff: 0.5165 - acc: 0.951 - ETA: 3:28 - loss: -0.9271 - dice_coeff: 0.5164 - acc: 0.951 - ETA: 3:27 - loss: -0.9272 - dice_coeff: 0.5164 - acc: 0.951 - ETA: 3:27 - loss: -0.9273 - dice_coeff: 0.5165 - acc: 0.951 - ETA: 3:27 - loss: -0.9270 - dice_coeff: 0.5165 - acc: 0.951 - ETA: 3:26 - loss: -0.9266 - dice_coeff: 0.5163 - acc: 0.951 - ETA: 3:26 - loss: -0.9264 - dice_coeff: 0.5162 - acc: 0.951 - ETA: 3:25 - loss: -0.9268 - dice_coeff: 0.5164 - acc: 0.951 - ETA: 3:25 - loss: -0.9269 - dice_coeff: 0.5164 - acc: 0.951 - ETA: 3:24 - loss: -0.9270

1195/1419 [========================>.....] - ETA: 2:30 - loss: -0.9288 - dice_coeff: 0.5173 - acc: 0.951 - ETA: 2:30 - loss: -0.9291 - dice_coeff: 0.5174 - acc: 0.951 - ETA: 2:29 - loss: -0.9291 - dice_coeff: 0.5174 - acc: 0.951 - ETA: 2:29 - loss: -0.9294 - dice_coeff: 0.5176 - acc: 0.951 - ETA: 2:29 - loss: -0.9297 - dice_coeff: 0.5177 - acc: 0.951 - ETA: 2:28 - loss: -0.9294 - dice_coeff: 0.5175 - acc: 0.951 - ETA: 2:28 - loss: -0.9297 - dice_coeff: 0.5177 - acc: 0.951 - ETA: 2:27 - loss: -0.9299 - dice_coeff: 0.5178 - acc: 0.951 - ETA: 2:27 - loss: -0.9298 - dice_coeff: 0.5177 - acc: 0.951 - ETA: 2:27 - loss: -0.9301 - dice_coeff: 0.5178 - acc: 0.951 - ETA: 2:26 - loss: -0.9302 - dice_coeff: 0.5178 - acc: 0.951 - ETA: 2:26 - loss: -0.9300 - dice_coeff: 0.5178 - acc: 0.951 - ETA: 2:25 - loss: -0.9295 - dice_coeff: 0.5176 - acc: 0.951 - ETA: 2:25 - loss: -0.9292 - dice_coeff: 0.5175 - acc: 0.951 - ETA: 2:24 - loss: -0.9296 - dice_coeff: 0.5177 - acc: 0.951 - ETA: 2:24 - loss: -0.9295

1329/1419 [===========================>..] - ETA: 1:34 - loss: -0.9298 - dice_coeff: 0.5180 - acc: 0.951 - ETA: 1:34 - loss: -0.9299 - dice_coeff: 0.5181 - acc: 0.951 - ETA: 1:33 - loss: -0.9297 - dice_coeff: 0.5180 - acc: 0.951 - ETA: 1:33 - loss: -0.9299 - dice_coeff: 0.5181 - acc: 0.951 - ETA: 1:33 - loss: -0.9299 - dice_coeff: 0.5180 - acc: 0.951 - ETA: 1:32 - loss: -0.9298 - dice_coeff: 0.5180 - acc: 0.951 - ETA: 1:32 - loss: -0.9300 - dice_coeff: 0.5181 - acc: 0.951 - ETA: 1:31 - loss: -0.9299 - dice_coeff: 0.5180 - acc: 0.951 - ETA: 1:31 - loss: -0.9297 - dice_coeff: 0.5179 - acc: 0.951 - ETA: 1:31 - loss: -0.9291 - dice_coeff: 0.5176 - acc: 0.951 - ETA: 1:30 - loss: -0.9286 - dice_coeff: 0.5173 - acc: 0.951 - ETA: 1:30 - loss: -0.9283 - dice_coeff: 0.5173 - acc: 0.951 - ETA: 1:29 - loss: -0.9286 - dice_coeff: 0.5174 - acc: 0.951 - ETA: 1:29 - loss: -0.9283 - dice_coeff: 0.5173 - acc: 0.951 - ETA: 1:28 - loss: -0.9278 - dice_coeff: 0.5170 - acc: 0.951 - ETA: 1:28 - loss: -0.9279

1419/1419 [==============================] - ETA: 37s - loss: -0.9265 - dice_coeff: 0.5165 - acc: 0.95 - ETA: 37s - loss: -0.9265 - dice_coeff: 0.5165 - acc: 0.95 - ETA: 37s - loss: -0.9263 - dice_coeff: 0.5165 - acc: 0.95 - ETA: 36s - loss: -0.9261 - dice_coeff: 0.5164 - acc: 0.95 - ETA: 36s - loss: -0.9264 - dice_coeff: 0.5165 - acc: 0.95 - ETA: 35s - loss: -0.9269 - dice_coeff: 0.5167 - acc: 0.95 - ETA: 35s - loss: -0.9266 - dice_coeff: 0.5166 - acc: 0.95 - ETA: 34s - loss: -0.9265 - dice_coeff: 0.5166 - acc: 0.95 - ETA: 34s - loss: -0.9260 - dice_coeff: 0.5163 - acc: 0.95 - ETA: 34s - loss: -0.9257 - dice_coeff: 0.5161 - acc: 0.95 - ETA: 33s - loss: -0.9260 - dice_coeff: 0.5163 - acc: 0.95 - ETA: 33s - loss: -0.9260 - dice_coeff: 0.5163 - acc: 0.95 - ETA: 32s - loss: -0.9264 - dice_coeff: 0.5164 - acc: 0.95 - ETA: 32s - loss: -0.9266 - dice_coeff: 0.5165 - acc: 0.95 - ETA: 31s - loss: -0.9265 - dice_coeff: 0.5165 - acc: 0.95 - ETA: 31s - loss: -0.9264 - dice_coeff: 0.5164 - acc: 0.

In [ ]:
def plot_training(history):
    acc = history.history['acc']
    print(acc)
    loss = history.history['loss']
    dice_coeff = history.history['dice_coeff']
    epochs = range(len(acc))

    plt.plot(epochs, acc, 'b.')
    plt.title('Training accuracy')
    plt.figure()
    plt.plot(epochs, loss, 'r-')
    plt.title('Training loss')
    plt.figure()
    plt.plot(epochs, dice_coeff, 'g.')
    plt.title('Training dice coeff')
    plt.figure()

    plt.show()
    
plot_training(history)